In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os,sys
sys.path.insert(0,"..")
sys.path.insert(0,"../..")
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import sys
import torch.nn.functional as F
import pandas as pd

In [3]:
import torchxrayvision as xrv

In [4]:
nn_model = xrv.models.DenseNet(weights="all")#.cuda()
nn_model.op_threshs = None # to prevent sigmoid

In [5]:
import torchvision, torchvision.transforms
transform = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop(),xrv.datasets.XRayResizer(224)])

In [6]:
# data is at `git checkout d0cb037cc863308a2f68780d9975275376f49a0b`
# https://github.com/ieee8023/covid-chestxray-dataset
path_to_covid_dataset = "../../../covid-chestxray-dataset"

In [7]:
dmerge = xrv.datasets.COVID19_Dataset(imgpath=path_to_covid_dataset + "/images",
                                      csvpath=path_to_covid_dataset + "/metadata.csv",
                                      transform=transform, views=["PA", "AP"])

In [8]:
dmerge

{'ARDS': {0.0: 393, 1.0: 15},
 'Bacterial': {0.0: 377, 1.0: 31},
 'COVID-19': {0.0: 96, 1.0: 312},
 'Chlamydophila': {0.0: 407, 1.0: 1},
 'E.Coli': {0.0: 408},
 'Fungal': {0.0: 395, 1.0: 13},
 'Influenza': {0.0: 407, 1.0: 1},
 'Klebsiella': {0.0: 401, 1.0: 7},
 'Legionella': {0.0: 404, 1.0: 4},
 'Lipoid': {0.0: 405, 1.0: 3},
 'MERS': {0.0: 408},
 'Mycoplasma': {0.0: 404, 1.0: 4},
 'No Finding': {0.0: 393, 1.0: 15},
 'Pneumocystis': {0.0: 395, 1.0: 13},
 'Pneumonia': {0.0: 15, 1.0: 393},
 'SARS': {0.0: 392, 1.0: 16},
 'Streptococcus': {0.0: 395, 1.0: 13},
 'Varicella': {0.0: 404, 1.0: 4},
 'Viral': {0.0: 75, 1.0: 333}}


COVID19_Dataset num_samples=408 views=['PA', 'AP']

In [9]:
dmerge.csv = dmerge.csv.join(pd.DataFrame(dmerge.labels, columns=["gt_" + path for path in dmerge.pathologies]))

In [10]:
scores = pd.read_csv(path_to_covid_dataset + "/annotations/covid-severity-scores.csv", skiprows=5)
scores = scores.set_index("filename")

In [11]:
scores

,geographic_mean,opacity_mean
filename,,
01E392EE-69F9-4E33-BFCE-E5C968654078.jpeg,6.0,4.0
03BF7561-A9BA-4C3C-B8A0-D3E585F73F3C.jpeg,2.7,2.0
1-s2.0-S0140673620303706-fx1_lrg.jpg,2.0,2.0
1-s2.0-S1684118220300608-main.pdf-001.jpg,3.7,3.0
1-s2.0-S1684118220300608-main.pdf-002.jpg,1.7,1.7
...,...,...
ryct.2020200028.fig1a.jpeg,2.0,1.7
ryct.2020200034.fig2.jpeg,1.3,1.3
ryct.2020200034.fig5-day0.jpeg,3.7,3.3


In [12]:
dmerge.csv = dmerge.csv.set_index("filename").join(scores, on="filename").reset_index()

In [13]:
dmerge.csv.geographic_mean[~dmerge.csv.geographic_mean.isna()].describe()

count    94.000000
mean      3.336170
std       2.369785
min       0.000000
25%       1.300000
50%       3.000000
75%       4.925000
max       8.000000
Name: geographic_mean, dtype: float64

In [14]:
dl = torch.utils.data.DataLoader(dmerge, batch_size=32, num_workers=2, pin_memory=True)

In [15]:
# d_feats = []
# d_preds = []
# d_pixels = []
# with torch.no_grad():
#     for i, batch in enumerate(dl):
#         imgs = batch["img"]#.cuda()
#         out = nn_model.features(imgs)
#         out = F.relu(out, inplace=True)
#         out = F.adaptive_avg_pool2d(out, (1, 1))
#         pred = nn_model.classifier(out.view(out.size(0), -1))
#         d_pixels.append(imgs)
#         d_feats.append(out.detach().cpu().numpy())
#         d_preds.append(pred.detach().cpu().numpy())
#         if i %10 == 0:
#             print(i)
# d_feats = np.concatenate(d_feats).reshape(-1,1024)
# d_preds = np.concatenate(d_preds)
# d_pixels = np.concatenate(d_pixels)

In [16]:
import pickle
#pickle.dump([d_feats, d_preds, d_pixels], open("d_feats.pkl","wb"))
d_feats, d_preds, d_pixels = pickle.load(open("d_feats.pkl","rb"))

In [17]:
dmerge.csv = dmerge.csv.join(pd.DataFrame(d_preds, columns=xrv.datasets.default_pathologies))

In [18]:
dmerge.csv["country"] = dmerge.csv.location.str.split(",").str[-1].str.strip()

In [19]:
continents = pd.read_csv("continents.csv.zip")
continents = continents[["name", "region", "sub-region"]]

In [20]:
dmerge.csv = dmerge.csv.join(continents.set_index("name"), on=["country"])

In [21]:
d_preds.shape

(408, 18)

In [22]:
d_feats.shape

(408, 1024)

In [23]:
import sklearn, sklearn.linear_model, sklearn.metrics, sklearn.model_selection, sklearn.neural_network
import scipy, scipy.stats, collections
def evaluate(data, labels, title, regions, test_region, plot=True, seed=0, method="linear", target_str="", usemlp=False):
    X = data
    y = labels.astype(float)
    
#     gss = sklearn.model_selection.GroupShuffleSplit(train_size=0.5,test_size=0.5, random_state=seed)
#     train_inds, test_inds = next(gss.split(X, y, groups))
    test_region_mask = (regions == test_region)
    train_inds = np.where(~test_region_mask)[0]
    test_inds = np.where(test_region_mask)[0]

    X_train, X_test, y_train, y_test = X[train_inds], X[test_inds], y.iloc[train_inds], y.iloc[test_inds]
    #print("X_train", X_train.shape, "X_test", X_test.shape)

    res = {}
    
    if method=="linear":
        model = sklearn.linear_model.LinearRegression()
    if (method=="logistic") and not usemlp:
        model = sklearn.linear_model.LogisticRegression()
    if usemlp:    
        model = sklearn.neural_network.MLPClassifier(random_state=seed)
        
        
    model = model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    pdiff = y_test - y_pred
    
    
    res["name"] = title
    res["test_region"] = test_region
    if method=="linear":
        res["R^2"] = sklearn.metrics.r2_score(y_test, y_pred)
        res["Correlation"] = scipy.stats.pearsonr(y_test,y_pred)[0]
        if np.isnan(res["Correlation"]): res["Correlation"] = 0
        res["MAE"] = sklearn.metrics.mean_absolute_error(y_test, y_pred)
        #res["MSE"] = sklearn.metrics.mean_squared_error(y_test, y_pred)
        res["# test samples"] = int(len(y_test))
        
    if method=="logistic":
        res["AUROC"] = sklearn.metrics.roc_auc_score(y_test, y_pred)
        res["AUPRC"] = sklearn.metrics.average_precision_score(y_test, y_pred)
        res["# test samples"] = collections.Counter(y_test == 1)
    
    if usemlp:
        #res["# params"] = "{}+{}".format(len(np.concatenate([a.flatten() for a in model.coefs_])), len(np.concatenate([a.flatten() for a in model.intercepts_])))
        res["# params"] = "{}".format(len(np.concatenate([a.flatten() for a in model.coefs_]))+len(np.concatenate([a.flatten() for a in model.intercepts_])))
        res["method"] = "MLP"
        res["name"] = res["name"] + " (MLP)"
    else:
        res["# params"] = "{}+1".format(len(model.coef_.flatten()))
        res["method"] = method

    
    if plot:
        fig, ax = plt.subplots(figsize=(6,4), dpi=100)
        for x,y,yp in zip(y_test,y_test,y_pred):
            plt.plot((x,x),(y,yp),color='red',marker='')

        pmax = int(np.max([y_pred.max(), y_test.max()]))+2
        plt.plot(range(pmax),range(pmax), c="gray", linestyle="--")
        plt.xlim(0,pmax-1)
        plt.ylim(0,pmax-1)

        plt.scatter(y_test, y_pred);
        plt.ylabel("Model prediction ($y_{pred}$)")
        plt.xlabel("Ground Truth ($y_{true}$)")
        plt.title(title);
        plt.text(0.01,0.97, "$R^2$={0:0.2f}".format(res["R^2"])+ "\n"+ 
                 "Correlation={0:0.2f}".format(res["Correlation"]), ha='left', va='top', transform=plt.gca().transAxes)

    return res#, pdiff


In [24]:
dmerge.csv.country.value_counts()

Italy             71
Germany           50
Spain             44
China             30
United Kingdom    28
Australia         26
USA               18
Hong Kong         14
Taiwan            14
Canada            12
South Korea        9
Portugal           8
Vietnam            6
Iran               6
Japan              6
Pakistan           5
Austria            3
Egypt              3
Singapore          3
Thailand           3
Philippines        3
Malta              2
Israel             2
Colombia           1
Belgium            1
Argentina          1
Russia             1
Sweden             1
Denmark            1
Greece             1
Turkey             1
Name: country, dtype: int64

In [25]:
dmerge.csv.region.value_counts()

Europe      211
Asia        102
Oceania      26
Americas     14
Africa        3
Name: region, dtype: int64

In [26]:
dmerge.csv["sub-region"].value_counts()

Southern Europe                    126
Eastern Asia                        73
Western Europe                      54
Northern Europe                     30
Australia and New Zealand           26
South-eastern Asia                  15
Northern America                    12
Southern Asia                       11
Western Asia                         3
Northern Africa                      3
Latin America and the Caribbean      2
Eastern Europe                       1
Name: sub-region, dtype: int64

In [27]:
# labels_mask = ~np.isnan(dmerge.csv.geographic_mean.astype(float))
# labels = dmerge.csv.geographic_mean[labels_mask]
# data = d_preds[labels_mask]
# data = data[:,[xrv.datasets.default_pathologies.index("Lung Opacity")]]
# for country in test_countries:
#     test_country_mask = dmerge.csv.country[labels_mask] == country
#     a = evaluate(data, labels,"Geographic Extent (Test:{})".format(country),test_country_mask = test_country_mask, seed=1)


In [28]:
# labels_mask = ~np.isnan(dmerge.csv.opacity_mean.astype(float))
# labels = dmerge.csv.opacity_mean[labels_mask]
# data = d_preds[labels_mask]
# data = data[:,[xrv.datasets.default_pathologies.index("Lung Opacity")]]
# for country in test_countries:
#     test_country_mask = dmerge.csv.country[labels_mask] == country
#     a = evaluate(data, labels,"Opacity (Test:{})".format(country),test_country_mask = test_country_mask, seed=0)


In [29]:
results = pd.DataFrame()

In [30]:
# target_str = "Geographic Extent"
# labels_mask = ~np.isnan(dmerge.csv.geographic_mean.astype(float))
# labels = dmerge.csv.geographic_mean[labels_mask]
# method = "linear"

In [31]:
target_str = "Opacity"
labels_mask = ~np.isnan(dmerge.csv.opacity_mean.astype(float))
labels = dmerge.csv.opacity_mean[labels_mask]
method = "linear"

In [32]:
# target_str = "Survival"
# labels_mask = dmerge.csv.survival.isin(["Y","N"]) & \
#               ~(dmerge.csv.intubation_present == "Y") & \
#               (dmerge.csv.offset.astype("float") > 0) & \
#               (dmerge.csv.offset.astype("float") < 8)
# labels = dmerge.csv.survival[labels_mask] == "Y"
# method = "logistic"

In [33]:
# target_str = "ICU Stay"
# labels_mask = dmerge.csv.went_icu.isin(["Y","N"]) & \
#               ~(dmerge.csv.in_icu == "Y") & \
#               ~(dmerge.csv.intubation_present == "Y") & \
#               (dmerge.csv.offset.astype("float") > 0) & \
#               (dmerge.csv.offset.astype("float") < 8)
# labels = (dmerge.csv.went_icu[labels_mask] == "Y")[labels_mask]
# method = "logistic"

In [34]:
# target_str = "intubated"
# labels_mask = dmerge.csv.intubated.isin(["Y","N"]) & \
#               ~(dmerge.csv.intubation_present == "Y") & \
#               (dmerge.csv.offset.astype("float") > 0) 
# labels = (dmerge.csv.intubated[labels_mask] == "Y")[labels_mask]
# method = "logistic"

In [35]:
# target_str = "COVID-19"
# labels_mask = ~dmerge.csv.finding.str.contains("todo") & \
#               (dmerge.csv.offset.astype("float") > 0) & \
#               (dmerge.csv.offset.astype("float") < 8) 
# labels = dmerge.csv.finding.str.contains("COVID-19")[labels_mask]
# method = "logistic"

In [36]:
# target_str = "Viral or Bacterial"
# labels_mask = ((dmerge.csv.gt_Bacterial == 1) | (dmerge.csv.gt_Viral == 1))& \
#               (dmerge.csv.offset.astype("float") > 0) & \
#               (dmerge.csv.offset.astype("float") < 8) 
# labels = (dmerge.csv.gt_Viral == 1)[labels_mask]
# method = "logistic"

In [37]:
labels_mask.sum()

94

In [38]:
dmerge.csv.region[labels_mask].unique()

array(['Asia', 'Americas', nan, 'Europe', 'Oceania'], dtype=object)

In [39]:
dmerge.csv["sub-region"][labels_mask].unique()

array(['South-eastern Asia', 'Eastern Asia', 'Northern America', nan,
       'Northern Europe', 'Southern Europe', 'Australia and New Zealand',
       'Western Asia', 'Southern Asia'], dtype=object)

In [40]:
groupby = "region"

In [41]:
test_regions = []
if method == "logistic":
    for region in dmerge.csv[groupby][labels_mask].dropna().unique():
        counts = labels[labels_mask][dmerge.csv[groupby][labels_mask] == region].value_counts()
        print(region, dict(counts))
        if len(counts) > 1 and counts.min() > 0:
            test_regions.append(region)        
if method == "linear":
    for region in dmerge.csv[groupby][labels_mask].dropna().unique():
        if (dmerge.csv[groupby][labels_mask] == region).sum() > 2:
            test_regions.append(region)       
test_regions

['Asia', 'Europe']

In [42]:
results = pd.DataFrame()

In [43]:
data = d_feats[labels_mask]
for test_region in test_regions:
    results = results.append(evaluate(data, labels, "Intermediate features", plot=False, 
                                      regions = dmerge.csv[groupby][labels_mask],
                                      test_region=test_region,
                                      target_str=target_str, 
                                      method=method), ignore_index=True)
    

In [44]:
data = np.zeros((data.shape[0],1))
for test_region in test_regions:
    results = results.append(evaluate(data, labels, "No data", plot=False, 
                                      regions = dmerge.csv[groupby][labels_mask],
                                      test_region=test_region,
                                      target_str=target_str, 
                                      method=method), ignore_index=True)

In [45]:
data = d_preds[labels_mask]
for test_region in test_regions:
    results = results.append(evaluate(data, labels, "18 outputs", plot=False, 
                                      regions = dmerge.csv[groupby][labels_mask],
                                      test_region=test_region,
                                      target_str=target_str, 
                                      method=method), ignore_index=True)

In [46]:
data = d_preds[labels_mask]
data = data[:,[xrv.datasets.default_pathologies.index("Lung Opacity"),
               xrv.datasets.default_pathologies.index("Pneumonia"),
               xrv.datasets.default_pathologies.index("Infiltration"),
               xrv.datasets.default_pathologies.index("Consolidation")]]

for test_region in test_regions:
    results = results.append(evaluate(data, labels, "4 outputs", plot=False, 
                                      regions = dmerge.csv[groupby][labels_mask],
                                      test_region=test_region,
                                      target_str=target_str, 
                                      method=method), ignore_index=True)

In [47]:
data = d_preds[labels_mask]
data = data[:,[xrv.datasets.default_pathologies.index("Lung Opacity")]]
for test_region in test_regions:
    results = results.append(evaluate(data, labels, "lung opacity output", plot=False, 
                                      regions = dmerge.csv[groupby][labels_mask],
                                      test_region=test_region,
                                      target_str=target_str, 
                                      method=method), ignore_index=True)

In [48]:
# data = d_pixels.reshape(d_pixels.shape[0],-1)[labels_mask]
# for test_region in test_regions:
#     for seed in range(3):
#         results = results.append(evaluate(data, labels, "Image pixels", plot=False, 
#                                           regions = dmerge.csv[groupby][labels_mask],
#                                           test_region=test_region,
#                                           target_str=target_str, 
#                                           method=method, 
#                                           seed=seed,
#                                           usemlp=True), ignore_index=True)

In [49]:
results

,# params,# test samples,Correlation,MAE,R^2,method,name,test_region
0,1024+1,36.0,5.973973e-01,1.387112,-0.412039,linear,Intermediate features,Asia
1,1024+1,41.0,7.598335e-01,1.019488,0.224734,linear,Intermediate features,Europe
2,1+1,36.0,2.614829e-16,1.301245,-0.120027,linear,No data,Asia
3,1+1,41.0,0.000000e+00,1.296917,-0.402600,linear,No data,Europe
4,18+1,36.0,5.687020e-01,1.021268,0.106629,linear,18 outputs,Asia
5,18+1,41.0,7.555287e-01,0.788424,0.479761,linear,18 outputs,Europe
6,4+1,36.0,7.451234e-01,0.797400,0.535371,linear,4 outputs,Asia
7,4+1,41.0,8.375430e-01,0.661321,0.690752,linear,4 outputs,Europe
8,1+1,36.0,7.384419e-01,0.834249,0.533505,linear,lung opacity output,Asia
9,1+1,41.0,8.394008e-01,0.694675,0.666877,linear,lung opacity output,Europe


In [50]:
print(results.round(2).to_latex(index=False))

\begin{tabular}{lrrrrlll}
\toprule
\# params &  \# test samples &  Correlation &   MAE &   R\textasciicircum 2 &  method &                   name & test\_region \\
\midrule
  1024+1 &            36.0 &         0.60 &  1.39 & -0.41 &  linear &  Intermediate features &        Asia \\
  1024+1 &            41.0 &         0.76 &  1.02 &  0.22 &  linear &  Intermediate features &      Europe \\
     1+1 &            36.0 &         0.00 &  1.30 & -0.12 &  linear &                No data &        Asia \\
     1+1 &            41.0 &         0.00 &  1.30 & -0.40 &  linear &                No data &      Europe \\
    18+1 &            36.0 &         0.57 &  1.02 &  0.11 &  linear &             18 outputs &        Asia \\
    18+1 &            41.0 &         0.76 &  0.79 &  0.48 &  linear &             18 outputs &      Europe \\
     4+1 &            36.0 &         0.75 &  0.80 &  0.54 &  linear &              4 outputs &        Asia \\
     4+1 &            41.0 &         0.84 &  0.66 &  0.69

In [51]:
if method=="linear":
    a = results.groupby(["name", "method"]).agg({
                                 'Correlation':['mean','std'],
                                 'R^2':['mean','std'],
                                 'MAE':['mean','std'],
                                # 'MSE':['mean','std']}
                                }).round(2)
if method=="logistic":
    a = results.groupby(["name", "method"]).agg({
                                 'AUROC':['mean','std'],
                                 'AUPRC':['mean','std']}).round(2)
a

Correlation         R^2         MAE      
                                    mean   std  mean   std  mean   std
name                  method                                          
18 outputs            linear        0.66  0.13  0.29  0.26  0.90  0.16
4 outputs             linear        0.79  0.07  0.61  0.11  0.73  0.10
Intermediate features linear        0.68  0.11 -0.09  0.45  1.20  0.26
No data               linear        0.00  0.00 -0.26  0.20  1.30  0.00
lung opacity output   linear        0.79  0.07  0.60  0.09  0.76  0.10

In [52]:
results[" "] = ""

In [53]:
if method=="linear":
    a = results.groupby([" "," ","name", "# params"]).agg({
                             'Correlation':['mean','std'],
                             'R^2':['mean','std'],
                             'MAE':['mean','std']}).round(2).sort_values(("MAE", "mean"))
if method=="logistic":
    a = results.groupby([" "," ","name", "# params"]).agg({
                                 'AUROC':['mean','std'],
                                 'AUPRC':['mean','std']}).round(2).sort_values(("AUROC", "mean"), ascending=False)
    

for c in a.columns.get_level_values(0).unique():
    a[c+"_"] =  a[(c, "mean")].apply("{0:0.2f}".format) + "$\pm$" + a[(c, "std")].apply("{0:0.2f}".format)
    del a[c]
    a[c] = a[c+"_"]
    del a[c+"_"]


In [54]:
a

Correlation             R^2  \
                                                                    
    name                  # params                                  
    4 outputs             4+1       0.79$\pm$0.07   0.61$\pm$0.11   
    lung opacity output   1+1       0.79$\pm$0.07   0.60$\pm$0.09   
    18 outputs            18+1      0.66$\pm$0.13   0.29$\pm$0.26   
    Intermediate features 1024+1    0.68$\pm$0.11  -0.09$\pm$0.45   
    No data               1+1       0.00$\pm$0.00  -0.26$\pm$0.20   

                                              MAE  
                                                   
    name                  # params                 
    4 outputs             4+1       0.73$\pm$0.10  
    lung opacity output   1+1       0.76$\pm$0.10  
    18 outputs            18+1      0.90$\pm$0.16  
    Intermediate features 1024+1    1.20$\pm$0.26  
    No data               1+1       1.30$\pm$0.00

In [55]:
print(", ".join(sorted(test_regions)))

Asia, Europe


In [56]:
", ".join(sorted(results.test_region.unique()))

'Asia, Europe'

In [57]:
print(target_str + "\\\\N=" + str(labels_mask.sum()))

Opacity\\N=94


In [58]:
print(a.to_latex(escape=False))

\begin{tabular}{lllllll}
\toprule
  &   &         &     &    Correlation &             R^2 &            MAE \\
  &   &         &     \\
  &   & name & # params &                &                 &                \\
\midrule
  &   & 4 outputs & 4+1 &  0.79$\pm$0.07 &   0.61$\pm$0.11 &  0.73$\pm$0.10 \\
  &   & lung opacity output & 1+1 &  0.79$\pm$0.07 &   0.60$\pm$0.09 &  0.76$\pm$0.10 \\
  &   & 18 outputs & 18+1 &  0.66$\pm$0.13 &   0.29$\pm$0.26 &  0.90$\pm$0.16 \\
  &   & Intermediate features & 1024+1 &  0.68$\pm$0.11 &  -0.09$\pm$0.45 &  1.20$\pm$0.26 \\
  &   & No data & 1+1 &  0.00$\pm$0.00 &  -0.26$\pm$0.20 &  1.30$\pm$0.00 \\
\bottomrule
\end{tabular}

